<a href="https://colab.research.google.com/github/yousrabougara/Text-Toxicity-Detection/blob/main/DL_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('toxicity_cleaned.csv',encoding ='UTF-8',engine='python',on_bad_lines='skip')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,b96754398592f2fd,respons imag conflict ________________________...,0,0,0,0,0,0
1,8426bd599b7ec559,i have to wait 8 more fuck day i am sick bulls...,1,1,1,0,1,0
2,825fead91a62a4ef,hey there i have notic huge addit furri articl...,0,0,0,0,0,0
3,f0b819fb8e844fa4,i swear to fuck christ leav me alon i t m y p ...,1,1,1,0,0,0
4,84fe00cd7a0d397c,might i remind everi one protect polici allow ...,0,0,0,0,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32548 entries, 0 to 32547
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             32548 non-null  object
 1   comment_text   32548 non-null  object
 2   toxic          32548 non-null  int64 
 3   severe_toxic   32548 non-null  int64 
 4   obscene        32548 non-null  int64 
 5   threat         32548 non-null  int64 
 6   insult         32548 non-null  int64 
 7   identity_hate  32548 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 2.0+ MB


In [ ]:
X = df['comment_text']
y = df.drop(columns=['comment_text','id'], axis=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(max_features = 5000, stop_words = 'english')
X = vector.fit_transform(df['comment_text'])
Y = df.drop(columns=['comment_text','id'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
#Splitting the New Dataset into Training and Testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
#Neural Network Model

model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),  # Specify input shape (number of features)
    Dropout(0.5),
    Dense(6, activation='sigmoid')  # 6 output units for multi-label classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # Binary cross-entropy for multi-label classification
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
train_model = model.fit(X_train.toarray(), y_train, epochs = 10, batch_size = 32, validation_split = 0.2)


Epoch 1/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8837 - loss: 0.4363 - val_accuracy: 0.9743 - val_loss: 0.2226
Epoch 2/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9673 - loss: 0.2191 - val_accuracy: 0.9743 - val_loss: 0.2001
Epoch 3/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9640 - loss: 0.1925 - val_accuracy: 0.9743 - val_loss: 0.1919
Epoch 4/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9676 - loss: 0.1759 - val_accuracy: 0.9743 - val_loss: 0.1884
Epoch 5/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9551 - loss: 0.1646 - val_accuracy: 0.9743 - val_loss: 0.1869
Epoch 6/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9478 - loss: 0.1553 - val_accuracy: 0.9741 - val_loss: 0.1869
Epoch 7/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9347 - loss: 0.1459 - val_accuracy: 0.9685 - val_loss: 0.1883
Epoch 8/10
651/651 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9226 - loss: 0.1433 - val_accuracy: 0.

In [ ]:
#Evaluating Model Accuracy On Test Data
"""Let's ensure that the model is not overfitting."""

loss, accuracy = model.evaluate(X_test.toarray(), y_test)
print(f"The Test Accuracy is: {accuracy}")

204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9512 - loss: 0.1986
The Test Accuracy is: 0.9480798840522766


In [ ]:
from sklearn.metrics import classification_report

#Predictions on Test Data
y_pred_prob = model.predict(X_test.toarray())
y_pred = (y_pred_prob > 0.5).astype(int)

#Classification Report
class_report = classification_report(y_test, y_pred)
print(class_report)

204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      2971
           1       0.51      0.19      0.28       282
           2       0.83      0.76      0.79      1621
           3       0.92      0.11      0.20        96
           4       0.71      0.69      0.70      1526
           5       0.63      0.31      0.41       264

   micro avg       0.81      0.73      0.77      6760
   macro avg       0.75      0.48      0.54      6760
weighted avg       0.80      0.73      0.76      6760
 samples avg       0.35      0.34      0.33      6760



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
#Saving the Keras Model
import pickle

with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vector, f)

model.save('toxic_comment_prediction_model.h5')

In [ ]:
 import pickle
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the label names (this should match the order of your output labels in the model)
label_names = [
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
]

# Loading TF-IDF Vectorizer
with open('/content/tfidf_vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

# Loading The Trained Model
loaded_model = load_model('/content/toxic_comment_prediction_model.h5')

# New comments to predict
new_comments = [
    "i will kill you ",
    "This is a very bad service.",
    "You’ve achieved so much!",
    "You are very stupid and mad.",
]

# Processing New Comments using the Loaded TF-IDF Vectorizer
processed_comment = loaded_vectorizer.transform(new_comments)

# Convert the sparse matrix to dense format (as the model expects dense input)
processed_comment_dense = processed_comment.toarray()

# Predicting using the Loaded Model
predictions = loaded_model.predict(processed_comment_dense)

# Thresholding predictions at 0.5 (for multi-label classification)
# If the predicted probability for a label is greater than 0.5, we consider it as 1 (toxic)
# Otherwise, it's 0 (not toxic)
thresholded_predictions = (predictions > 0.5).astype(int)

# Print the results with label names and 0/1 predictions
for comment, prediction in zip(new_comments, thresholded_predictions):
    print(f"Comment: {comment}")
    for label, is_toxic in zip(label_names, prediction):
        print(f"  {label:<15} {is_toxic}")
    print("\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Comment: i will kill you 
  toxic           1
  severe_toxic    0
  obscene         0
  threat          1
  insult          0
  identity_hate   0


Comment: This is a very bad service.
  toxic           1
  severe_toxic    0
  obscene         0
  threat          0
  insult          0
  identity_hate   0


Comment: You’ve achieved so much!
  toxic           0
  severe_toxic    0
  obscene         0
  threat          0
  insult          0
  identity_hate   0


Comment: You are very stupid and mad.
  toxic           1
  severe_toxic    0
  obscene         0
  threat          0
  insult          1
  identity_hate   0


